# SLU14: Model complexity and overfitting -- Exercises


---
*Exercises are graded unless otherwise indicated.*

In [ ]:
import inspect
import base64
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from utils import expand_dataset

from sklearn.datasets import load_boston
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import mutual_info_regression

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import json
from hashlib import sha1 # just for grading purposes

def _hash(obj):
    if type(obj) is not str:
        obj = json.dumps(obj)
    return sha1(obj.encode()).hexdigest()


As a junior data scientist, you are hired by a real estate company to work on their data. The first task they assign to you is to build a predictor of house prices based on a dataset they have. Now, being junior, you don't ask many questions about the purpose of this task and want to apply some of the stuff you've learned as quickly as possible so you dive right into the code.


### Data

For this exercise notebook we'll be working with a house price dataset containing several different features related to the house being sold and the seller. Start by loading the data:

In [ ]:
# This function is designed to be used in all the exercises

def load_dataset():
    # Loads house prices dataset
    df = pd.read_csv("data/house-prices.csv")
    
    return df

In [ ]:
# Load data
df = load_dataset()
df.head()


In [ ]:
df.shape

## Exercise 1 - complex models 


After you are briefed on the problem, and feeling already short on time, you decide to start by taking all your data and throwing it at a model. After all, with all the fancy algorithms in data science, it's easy to try and, *who knows*, you might even get a good performance from the start and even improve it further with parameter tuning and all that *fancy data science magic*.


<img alt="xkcd_shuffle_data_pile" src="media/xkcd_shuffle_data_pile.png" width="300">


### Exercise 1.1 

As a first step, to get the data into your model, you notice that you have a lot of categorical values. Knowing that most models need them to be encoded somehow, you keep it simple and decide to convert each categorical column into a set of dummy variables expanding on each category. 

Implement below the function to do so:

In [ ]:
def encode_categorical(df): 
    """ 
        Takes a dataframe and encodes all its categorical values
        into indicator variables
    
    Args:
        df (pd.DataFrame): the input DataFrame

    Returns: df
        df (pd.DataFrame): the dataframe with all categorical values encoded as 
            new variables
        
    """    
    # YOUR CODE HERE
    raise NotImplementedError()

    return df_encoded 


Validate your functions in the tests below:

In [ ]:
df = load_dataset()
df_encoded = encode_categorical(df)

assert df_encoded.shape == (1460, 1677)
assert "BsmtUnfSF" in df_encoded.columns
assert df_encoded["Neighborhood_Gilbert"][1042] == 0
assert df_encoded["Condition1_Artery"][100] == 0


And check your new dataframe below:

In [ ]:
df_encoded.head()

### Exercise 1.2

With your prepared data proceed to training a quick baseline. For that purpose you chose to use a decision tree regressor with max_depth of 7 and min_sample_split of 30. 


Create the function to produce this baseline below:

In [ ]:
def decision_tree_regression(X, y): 
    """
    Fit a DecisionTreeRegressor with `max_depth` of 7, and `min_samples_split` 
    of 30 on the provided data
    
    **For reproducibility: Use random state of 42 always**

    Args:
        X (pd.DataFrame): the input DataFrame
        y (pd.Series): the target labels

    Returns: estimator
        estimator (DecisionTreeRegressor): fitted estimator        
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return dt 


Validate the decision tree you've trained on tests below:

In [ ]:
df = load_dataset()
df_encoded = encode_categorical(df)

X = df_encoded.drop('SalePrice', axis=1)
y = df_encoded['SalePrice']

dt = decision_tree_regression(X, y)

assert dt.max_depth == 7
assert dt.min_samples_split == 30
assert dt.n_features_ == 1676

preds = dt.predict(X)

np.testing.assert_almost_equal(preds[100], 193.0600, 2)
np.testing.assert_almost_equal(preds[1042], 164.8525, 2)


### Exercise 1.3

Finally, you want to evaluate performance on your baseline, and so you create a small function to produce the two regression metrics you want to use: **mean squared error** and **r2 score**.

Implement it below:

In [ ]:
def regresion_metrics(y, preds): 
    """
    Return two regression metrics computed with the provided predictions
    `preds` and the true values `y`: mean squared error and R squared
    
    Args:
        preds (pd.Series): the predictions we want to assess
        y (pd.Series): the true labels

    Returns: mse, r2
        mse (float): mean squared error of predicitons     
        r2 (float): r-squared value of predicitons        
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return mse, r2 



In [ ]:
df = load_dataset()
df_encoded = encode_categorical(df)

X = df_encoded.drop('SalePrice', axis=1)
y = df_encoded['SalePrice']

dt = decision_tree_regression(X, y)
preds = dt.predict(X)

mse, r2 = regresion_metrics(y, preds)

np.testing.assert_almost_equal(mse, 716.2435, 2)
np.testing.assert_almost_equal(r2, 0.8996, 2)



Even though we got a pretty big Mean squared error, which can also relate to the big values we're handling, we get a pretty good R-squared value. 

However, you remember something somebody told you about how every time you evaluate on the data you used to train you lose the ability to tell whether you are overfitting the data you happen to have at hand. So you decide to also apply a held-out test set and check the results:



In [ ]:
df = load_dataset()
df_encoded = encode_categorical(df)
print("Number of features: {}".format(len(df_encoded.columns)))

X = df_encoded.drop('SalePrice', axis=1)
y = df_encoded['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

dt = decision_tree_regression(X_train, y_train)

preds = dt.predict(X_test)
mse, r2 = regresion_metrics(preds, y_test)

print("Mean Squared Error: {}".format(mse))
print("R-squared: {}".format(r2))


### Exercise 1.4

Not so great comparing to training, but also not incredibly bad. Now let's plot the decision tree used for this problem. Implement below a function plotting the decision tree:

- Showing all labels
- Not showing impurity labels
- Do not change the display of ‘values’ and/or ‘samples’ to be proportions

Optional (for aesthetics):
- Fill the nodes with `filled=True`
- Use `rounded=True` to have nice rounded corners


In [ ]:
def plot_dt_regressor(dt, feature_names): 
    """
    Generate plot of decision tree given the estimator and its 
    feature names. 

    Args:
        dt (DecisionTreeRegressor): fitted decision tree estimator        
        feature_names (list): list of feature names

    Returns: plot_annots
        plot_annots (list): plot annotations     

    """
    plt.subplots(figsize=(100, 20))  # this helps you control the figure size 

    # YOUR CODE HERE
    raise NotImplementedError()
    
    return plot_annots 


In [ ]:
df = load_dataset()
df_encoded = encode_categorical(df)

X = df_encoded.drop('SalePrice', axis=1)
y = df_encoded['SalePrice']

dt = decision_tree_regression(X, y)
tree_plot = plot_dt_regressor(dt, X.columns)

sig = inspect.signature(plot_dt_regressor)
assert set(sig.parameters.keys()) == {'dt', 'feature_names'}
all_text = ''.join([tree_plot[i].get_text() for i in range(len(tree_plot))])
assert 'OverallQual' in all_text

first_node_feature = tree_plot[0].get_text().split('<')[0].strip()
assert _hash(first_node_feature) == '0204560528837e345e60ecc1eccf33d641a30686'
assert len(tree_plot) == 107

node_10 = tree_plot[10].get_text()
assert 'value = 119.154' in node_10
assert 'BsmtFinSF1' in node_10, node_10


Check the feature names in the plot against the feature names in the dataframe you used:

In [ ]:
df_encoded

### Exercise 1.5

*Hint: To check the decision tree image better, you can save it and open in your own viewer of preference or you double click on the image and it will zoom in)*

You look into your process, dataframe and tree with a senior colleague. What is he most likely to tell you? 

* A) Great job, this seems like a great starter model! 
* B) You clearly need to prune your decision tree
* C) What is a decision tree?
* D) Have you looked into the dataset? There's some pretty random features here...


In [ ]:
# answer = 'A' or 'B' or 'C' or 'D'

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert base64.b64encode(answer.encode()) == b'RA=='


# Exercise 2 - find the nonsense 

### Exercise 2.1

After this first trial, you proceed with a more methodical approach. There are a few features which should obviously not influence the price and might be just adding unnecessary complexity to the model. In particular, there are two obvious ones that you should be able to identify and discard.

Can you find them?

Write below which columns (from the initial dataframe) you would remove.


_Hint #1: Look into your dataframe after encoding the categorical features. What feature is exploding the number of variables?_  

_Hint #2: What other non-categorical column has values that equal the amount of rows we have?_  

In [ ]:
# Example answer:
# nonsense_features = ['SomeFeature', 'SomeOtherFeature']

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert _hash(sorted(nonsense_features)) == 'a9c3a5038268a67480b94c5206dbb7ef3c54b2d1'


In [ ]:
df = load_dataset()
df_removed = df.drop(nonsense_features, axis=1)

df_encoded = encode_categorical(df_removed)

df_encoded.head()

Great job! That was a reduction of 1677 to 217 columns, or in terms of model, a reduction from 1677 to 217 features. 

In [ ]:
X = df_encoded.drop('SalePrice', axis=1)
y = df_encoded['SalePrice']
print("Number of features: {}".format(len(df_encoded.columns)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

dt = decision_tree_regression(X_train, y_train)

preds = dt.predict(X_train)
mse, r2 = regresion_metrics(preds, y_train)

print("Mean Squared Error (train): {}".format(mse))
print("R-squared (train): {}".format(r2))

preds = dt.predict(X_test)
mse, r2 = regresion_metrics(preds, y_test)

print("Mean Squared Error (test): {}".format(mse))
print("R-squared (test): {}".format(r2))

tree_plot = plot_dt_regressor(dt, X.columns)


We've dropped more than a thousand features and still get the same performance. Even though, depending on the models, these features may generate more or less noise, this shows how important it is to understand the data before you throw it into your model.

### Exercise 2.2

Another way that you can understand the data is by looking into the decision tree image and think about the decisions it's making. If you were to "guess" the most important feature given the decision tree, which one would you pick? 

* A) YearRemodAdd
* B) OverallQual
* C) WoodDeckSF
* D) CentralAir_Y

*Hint: To check the decision tree image better, you can save it and open in your own viewer of preference or you double click on the image and it will zoom in)*

In [ ]:
# answer = 'A' or 'B' or 'C' or 'D'

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert base64.b64encode(answer.encode()) == b'Qg=='

### Exercise 2.3

Let's continue on these lines and find other features that do not have importance in this dataset. Use the `mutual_information` method and find all of the features with 0.0 mutual information with the target:

*Hint: You have used the `mutual_info_classif` for classification. There is an analogous function for regression in sklearn*

In [ ]:
def zero_mutual_information_features(X, y): 
    """
    Use the mutual information method and return all feature names
    where the mutual information is zero.

    **For reproducibility: Use random state of 42 always**

    Args:
        X (pd.DataFrame): the input DataFrame
        y (pd.Series): the target labels

    Returns: plot_annots
        features (list): list of feature names where mutual information is zero     

    """
    # Compute mutual information of the dataset features
    # Hint: use can use the sklearn function here
    # mutual_info = ...
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Extract the feature names where the mutual information is zero
    # features = ...
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return features 




In [ ]:
df = load_dataset()
df_removed = df.drop(nonsense_features, axis=1)
df_encoded = encode_categorical(df_removed)

X = df_encoded.drop('SalePrice', axis=1)
y = df_encoded['SalePrice']

bottom_mut_info_features = zero_mutual_information_features(X, y)

assert _hash(sorted(bottom_mut_info_features)) == '9c41aa9d5505800207c36f3cb3ecdf9c1d58ea15'


If we now drop these features we should achieve similar values on our data. Check this below:

In [ ]:
df = load_dataset()
df_removed = df.drop(nonsense_features, axis=1)
df_encoded = encode_categorical(df_removed).drop(bottom_mut_info_features, axis=1)

print("Number of features: {}".format(len(df_encoded.columns)))

X_train, X_test, y_train, y_test = train_test_split(
    df_encoded.drop('SalePrice', axis=1), 
    df_encoded['SalePrice'], 
    test_size=0.3, 
    random_state=42
)

dt = decision_tree_regression(X_train, y_train)

preds = dt.predict(X_train)
mse, r2 = regresion_metrics(preds, y_train)

print("Mean Squared Error (train): {}".format(mse))
print("R-squared (train): {}".format(r2))

preds = dt.predict(X_test)
mse, r2 = regresion_metrics(preds, y_test)

print("Mean Squared Error (test): {}".format(mse))
print("R-squared (test): {}".format(r2))

In this case, our training values are not too different, but our test values decreased. So what's that all about? If these features brought nothing to the table when it came to predicting our target, shouldn't the performance stayed the same?

There are a couple of things to take into consideration here as we've been using a fixed model with some random parameters we've chosen to keep it simple:

* The parameters may not be adequate and may be leading to overfitting
* These "random" features may actually have been having a regularization effect on the training, preventing it to overfit too much to other features containing more information
* We may just have an unlucky split - methods like using a validation set to optimize parameters and then test on a held-out set or cross-validating with k-fold can help you get more reliable values 


As stated before not all models suffer equally from adding unnecessary features, but we can't stress enough how important it is that you make sense of your data and avoid delegating the work to your model. Hopefully if you get a real life project you'll start by exploring the features first and then iterating on the model.

Let's continue now with some exercises on model specific feature importance!

# Exercise 3: model specific feature importances

We'll now see some examples of model-specific feature importances. For this purpose, we'll leave aside the decision tree example and focus on two other estimators - linear and non-linear.

### Exercise 3.1

Start by implementing a LinearRegression model, fit it to our data, and extract the information about feature importances:


In [ ]:
def linear_regression_feat_importances(X, y): 
    """
    Fit a linear regression on the provided data and return the top 5 
    most important features

    Args:
        X (pd.DataFrame): the input DataFrame
        y (pd.Series): the target labels

    Returns: top_5_features
        top_5_features (list): list of 5 feature names of most importance    
    """
    # Fit a linear regression estimator
    # lr = ...
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Extract the top 5 features of the model
    # top_5_features = ...
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return top_5_features 


Test your function below:

In [ ]:
df = load_dataset()

# We're constraining the dataset to avoid precision errors
X = encode_categorical(
    df[
        [
            'LotArea', 
            'LotShape', 
            'LandContour', 
            'Utilities', 
            'OverallQual', 
            '1stFlrSF', 
            '2ndFlrSF',  
            'BsmtUnfSF', 
            'TotalBsmtSF',
            'Utilities',
            'LotConfig',
            'HouseStyle',
            'OverallCond',
            'YearBuilt',
            'YrSold',
            'MoSold',
            'GarageCars',
            'GarageArea',
            'SaleType',
            'SaleCondition',
            'Fireplaces',
            'BldgType',
            'BsmtFinSF1',
            'BsmtFinSF2',
            'Heating',
            'CentralAir',
            'LowQualFinSF',
            'HeatingQC',
            'GrLivArea',
            'BsmtFullBath'
        ]
    ]
)
y = df['SalePrice']

rs = RobustScaler()  # just scaling, because I'm nice. 
X_normed = pd.DataFrame(rs.fit_transform(X), columns=X.columns)

lr_features = linear_regression_feat_importances(X_normed, y)

assert _hash(sorted(lr_features)) == '6b99f61bb095ef08521ac22ea2fd0f53cd7e8f4e'

print("Top features for linear regression: {}".format(sorted(lr_features)))


### Exercise 3.2

Now let's do the same thing for a non-linear model. Train a Random Forest, with the following parameters: 

* n_estimators = 40 
* max_depth = 3
* min_samples_split = 20 
* random_state = 42
* n_jobs = -1  (optional, but speeds things up)

Then use it to get feature importances. Use the non-normalized features. As before, get the top 5 features by importance.

In [ ]:
def random_forest_feat_importances(X, y): 
    """
    Fit a random forest regressor on the provided data and return the top 5 
    most important features. Use 40 estimators, a max depth of 3 and a min
    sample split of 20

    **For reproducibility: Use random state of 42 always**

    Args:
        X (pd.DataFrame): the input DataFrame
        y (pd.Series): the target labels

    Returns: top_5_features
        top_5_features (list): list of 5 feature names of most importance    
    """
    # Fit a random forest regressor estimator
    # lr = ...
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Extract the top 5 features of the model
    # top_5_features = ...
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return top_5_features 



In [ ]:
df = load_dataset()
df_removed = df.drop(nonsense_features, axis=1)

df_encoded = encode_categorical(df_removed)

X = df_encoded.drop('SalePrice', axis=1)
y = df_encoded['SalePrice']

rf_features = random_forest_feat_importances(X, y)

assert _hash(sorted(rf_features)) == 'f4cb12310d33a0a0fc081d61b03ce96dcb98b2e2'

print("Top features for random forest regressor: {}".format(sorted(rf_features)))


Seeing the features of most importance to your model is another way that can help you understand the data better and iterate on the best solution for your problem.

By now, you've seen hopefully that feature selection is a useful tool to understand and iterate in your models, in particular it allows us to focus on what matters most and avoid overly complex models. Let's now jump into our final topic - regularization.

# Exercise 4: Regularization

Finally, we'll practice the regularization techniques you've learned for regression. Throughout the learning notebooks, you've seen different loss functions that can be used for regularization. We're going to explore L1, but you should be able to follow similar procedures later on for either of the other two.

For this purpose, we'll use another house price dataset - the Boston housing dataset. Load the data below:


In [ ]:
# This function is designed to be used in the following exercises

def load_boston_dataset():
    # Loads boston house prices dataset
    boston_data = load_boston()
    df_boston = pd.DataFrame(boston_data.data,columns=boston_data.feature_names)
    df_boston['target'] = pd.Series(boston_data.target)
    
    return df_boston

In [ ]:
# Load data
boston_data = load_boston_dataset()
boston_data.head()

In particular we'll try to check the relation between the LSTAT variable and the target - the house price. Let's start by using polynomial features and fitting a linear regressor:

In [ ]:
df_boston = load_boston_dataset()
df_train = df_boston.set_index('LSTAT', drop=False).sort_index()

X = df_train[['LSTAT']]
y = df_train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=111)

X_poly = expand_dataset(X_train, n_expansions=10, feature_name='LSTAT')
X_poly_test = expand_dataset(X_test, n_expansions=10, feature_name='LSTAT')

lr = LinearRegression()
lr.fit(X_poly, y_train)
preds_train = lr.predict(X_poly)
preds_test = lr.predict(X_poly_test)

X_train_plot, preds_plot = zip(*sorted(zip(X_train.values, preds_train)))

mse = mean_squared_error(y_train, preds_train)
print("Mean squared error (train): {}".format(mse))

r2 = r2_score(y_train, preds_train)
print("R2 score (test): {}".format(r2))

mse = mean_squared_error(y_test, preds_test)
print("Mean squared error (test): {}".format(mse))

r2 = r2_score(y_test, preds_test)
print("R2 score (test): {}".format(r2))

plt.figure(figsize=(5,5))
plt.scatter(X_train, y_train, c='orange', s=5, label="Original data")
plt.plot(X_train_plot, preds_plot, c='blue', label="Polynomial Regression")
plt.title("Polynomial regression with 10 degrees")
plt.legend(loc="best")
plt.xlabel('LSTAT')
plt.ylabel('target')
plt.show();

Seems a bit overfitted. 

### Exercise 4.1

So let's try some regularization techniques. First, remember the L1 loss:

$$J_{L_1} = \frac{1}{N} \sum_{n=1}^N (y_n - \hat{y}_n)^2 + \lambda_1 \sum_{k=1}^K \left|\beta_k\right|$$

Start by implementing it in the function below:

In [ ]:
def l1_loss(y, y_hat, betas, lamb):
    """
    Implement the L1-loss function 

    Args: 
        y (np.array): numpy array with shape (num_observations,)
            The targets.
        y_hat (np.array): numpy array with shape (num_observations,)
            The predictions.
        betas (np.array): numpy array with shape (num_features+1,)
            The parameters of your regression model. 
            The first value is the intercept and the 
            remaining ones are the feature coefficients.
        lamb (float): The strength of the L1 regularizer.
            
    Returns:
        L (float): loss with L1-regularization
    """
        
    # Compute the mean squared error loss part of the general loss function.
    # Hint: use can use the sklearn function here
    # mse = ...
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Compute the L1 part of the general loss function.
    # l1 = ...
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Compute the total loss by combining the parts.
    # L = ...
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return L

Test your loss function below:

In [ ]:
loss1 = l1_loss(
    np.array([0.89305005, 0.53765939, 0.58379047, 0.39355386, 0.30221551, 0.18592352, 0.4025869 , 0.63137406]), 
    np.array([0.23556499, 0.5899186 , 0.77827108, 0.57362455, 0.04727258, 0.99407257, 0.21856487, 0.25006812]), 
    np.array([0.45836509, 0.78367697, 0.6914712]), 
    1
)
np.testing.assert_almost_equal(loss1, 1.6504, 3)

loss1 = l1_loss(
    np.array([0.89305005, 0.53765939, 0.58379047, 0.39355386, 0.30221551, 0.18592352, 0.4025869 , 0.63137406]), 
    np.array([0.23556499, 0.5899186 , 0.77827108, 0.57362455, 0.04727258, 0.99407257, 0.21856487, 0.25006812]), 
    np.array([0.45836509, 0.78367697, 0.6914712]), 
    6
)
np.testing.assert_almost_equal(loss1, 9.0262, 3)


Now let's apply it to our subset of the data and the linear regression we've trained:

In [ ]:
df_boston = load_boston_dataset()
df_train = df_boston.set_index('LSTAT', drop=False).sort_index()

X = df_train[['LSTAT']]
y = df_train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=111)

X_poly = expand_dataset(X_train, n_expansions=10, feature_name='LSTAT')
X_poly_test = expand_dataset(X_test, n_expansions=10, feature_name='LSTAT')

lr = LinearRegression()
lr.fit(X_poly, y_train)
preds_train = lr.predict(X_poly)
preds_test = lr.predict(X_poly_test)

betas = np.append(lr.intercept_, lr.coef_)

l1_1 = l1_loss(y_train, preds_train, betas, 1)
print("L1 train (beta=1): {}".format(l1_1))

l1_5 = l1_loss(y_train, preds_train, betas, 5)
print("L1 train (beta=5): {}".format(l1_5))


It shouldn't surprise you that the loss gets larger by adding the regularization factor, as we let the linear regression train with focus solely on the mean squared error loss and our coefficients are probably somewhat large. Plot them below:

In [ ]:
plt.plot(range(len(lr.coef_)), [abs(coef) for coef in lr.coef_], marker='o', markerfacecolor='r')
plt.xlabel('Coefficient')
plt.ylabel('Coef. magnitude')
plt.title("Coefficient magnitude (no regularization)")
plt.show();

### Exercise 4.2

Now let's see what would happen if you would have trained using this regularized loss from the start. 

Implement below the function that fits a regressor using the l1_loss (*Hint: sklearn offers and out of the box estimator for this that you should use*)


In [ ]:
def l1_loss_lr(X, y, alpha=0.1):
    """
    Fit a linear regressor using the L1-loss with normalize=True and return it 

    Args:
        X (pd.DataFrame): the input DataFrame
        y (pd.Series): the target labels
        alpha (float): the L1 coefficient

    Returns: l1_estimator
        l1_estimator: fitted estimator        
    """
        
    # lasso_lr = ...
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return l1_estimator


Test the function below:

In [ ]:
df_boston = load_boston_dataset()
df_train = df_boston.set_index('LSTAT', drop=False).sort_index()

X = df_train[['LSTAT']]
y = df_train['target']

X_poly = expand_dataset(X, n_expansions=10, feature_name='LSTAT')
lr_l1 = l1_loss_lr(X_poly, y, alpha=0.05)

preds_train = lr_l1.predict(X_poly)

np.testing.assert_almost_equal(preds_train[1], 31.0376, 2)
np.testing.assert_almost_equal(preds_train[41], 29.0328, 2)
np.testing.assert_almost_equal(preds_train[232], 24.3181, 2)
np.testing.assert_almost_equal(preds_train[312], 21.7428, 2)


Now we'll apply this regressor to your data in the same way we did before:

In [ ]:
df_boston = load_boston_dataset()
df_train = df_boston.set_index('LSTAT', drop=False).sort_index()

X = df_train[['LSTAT']]
y = df_train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=111)

X_poly = expand_dataset(X_train, n_expansions=10, feature_name='LSTAT')
X_poly_test = expand_dataset(X_test, n_expansions=10, feature_name='LSTAT')

lr_l1 = l1_loss_lr(X_poly, y_train, alpha=0.0001)
preds_train = lr_l1.predict(X_poly)
preds_test = lr_l1.predict(X_poly_test)

betas = np.append(lr_l1.intercept_, lr_l1.coef_)

mse = mean_squared_error(y_train, preds_train)
print("Mean squared error (train): {}".format(mse))

r2 = r2_score(y_train, preds_train)
print("R2 score (test): {}".format(r2))

mse = mean_squared_error(y_test, preds_test)
print("Mean squared error (test): {}".format(mse))

r2 = r2_score(y_test, preds_test)
print("R2 score (test): {}".format(r2))



We get much more stable values! Great! Finally, we can plot the coefficients for this model: 

In [ ]:
plt.plot(range(len(lr_l1.coef_)), [abs(coef) for coef in lr_l1.coef_], marker='o', markerfacecolor='r')
plt.xlabel('Coefficient')
plt.ylabel('Coef. magnitude')
plt.title("Coefficient magnitude (with regularization)")
plt.show();

They are much lower, which typically can help prevent overfitting, as you've seen before.

And that's a wrap!

Congratulations on getting to the end of this exercise notebook. We hope you understood the importance of looking into your data and understanding it, making informed decisions on what features to use and what features to discard. Additionally, you should always keep in mind that starting simple and building from there is a much better approach than choosing overly complex approaches that will become harder to debug.


See you on the next unit! 

</br> 

<img alt="affordable_house" src="media/affordable-house.jpg" width="500">

